In [ ]:
# import custom modules
from custom_libs import db
from custom_libs import preprocessing
from custom_libs import classification
from custom_libs import plotting
from custom_libs import utils
from custom_libs import dump

## Loading del dataset e Preprocessing

In [ ]:
df = db.get_dataset("McDonald_s_Reviews")
preprocessing.add_id_column(df, columns=["latitude", "longitude"])
preprocessing.add_rating_number_column(df)
df.head(2)

In [ ]:
proprocessing_function = preprocessing.preprocess_text
preprocessing.preprocess_dataframe(df, 'review', proprocessing_function)
df.head(2)

## Cose delle Classificazioni

In [ ]:
model = dump.load_model("svc_model_chat")
vectorizer = dump.load_model("svc_vectorizer_chat")

In [ ]:
res = model.predict(vectorizer.transform(df['review_clean']))
df['sentiment_our'] = res
df.head(2)

In [ ]:
classification.append_sentiment_for_each_row(df, 'review_clean', new_column_name='sentiment_auto')
df[['review_clean', 'sentiment_auto', 'sentiment_our']]

In [ ]:
# select only the rows where the sentiment_our and sentiment_auto is different
df2 = df[df['sentiment_our'] != df['sentiment_auto']]
df2.shape

In [ ]:
# get the rows with sentiment_our=Positive and sentiment_auto=Negative or viceversa
df2 = df2[(df2['sentiment_our'] == classification.Sentiment.POSITIVE) & (df2['sentiment_auto'] == classification.Sentiment.NEGATIVE) | (df2['sentiment_our'] == classification.Sentiment.NEGATIVE) & (df2['sentiment_auto'] == classification.Sentiment.POSITIVE)]
df2.shape

In [ ]:
def plot_sentiment_word_clouds(df):
    for sentiment in classification.Sentiment.get_all():
        df_sentiment = df[(df['sentiment']==sentiment)]
        plotting.plot_word_cloud(df_sentiment, sentiment, 'review_clean')

# plot_sentiment_word_clouds(df)

In [ ]:
# non cancellare per ora questo commento: [30.460718, -97.792874]  max_distance=100
current_position = [30.460718, -97.792874]
best_rated_restaurant = utils.select_best_restaurant_from_stars(df, current_position, max_distance=100)

In [ ]:
best_feeling_restaurant = utils.select_best_restaurant_from_sentiment(df, current_position, max_distance=100)
best_feeling_restaurant